#Problem Statement

We have a huge number of comments from YouTube for a latest trailer from a worldwide production house, you as an AI
service provider are supposed to analyse all the comments on that trailer, get the sentiment and the score, and give
a consolidated report for that trailer about how it might perform on the box office.


In [ ]:
#Tools and high level steps that will be used in the project.
"""
1. Get your comments from the Youtube trailer. One option would manually getting the comments, get them directly from the API, you can save them in a file and
load it in your colab.

2.Setup your colab to do the job for you, you will need to install the required libraries.
PyTorch - torch
HuggingFace - transformers
NLTK - nltk
VADER - sentiment.vader

"""

In [ ]:
"""
STEPS :-

1.Do all the necessary imports
2.create a function for removing stop words
3.create a function to calculate the sentiment score and the sentiment(positive/negative)
4.Loop through the Comments that you will get from your input excel file
4.5 Seggreate the words into positive and negative, so you can make a word cloud at the end
5.Calculate all the sentiments in loop and return only one final result

"""

In [ ]:
!pip install torch

In [ ]:
!pip install transformers


In [ ]:
!pip install nltk

In [ ]:
!pip install vaderSentiment

In [ ]:
import pandas as pd
df = pd.read_excel("/content/SnowWhite Comments.xlsx")



In [ ]:
comments = []
comments = df["Comments"].tolist()

In [ ]:
for comment in comments:
  print(comment)
  print("===")
# print(comments)

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

sia = SentimentIntensityAnalyzer()

#stopwords - WOrds that help make up a sentence but don't have their own meaning
#it, they, them, what, am, I
comment_score1 = sia.polarity_scores("I am Very happy with this movie.")
print(comment_score1)


comment_score2 = sia.polarity_scores("I am extremely sad today.")
print(comment_score2)


comment_score3 = sia.polarity_scores("This is an amazing and outstanding movie.")
print(comment_score3)


comment_score4 = sia.polarity_scores("The first part was amazing and outstanding, but this part sucks.")
print(comment_score4)

In [ ]:
#for classifying my comments into positive and negative, I will be using an AI model
#from HuggingFace to get the score of the sentence, we will go with nltk/vader sentiment.

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
nltk.download("punkt_tab")

In [ ]:
sentence = "Today is a very warm and sunny day, I would like to go out and play football with my friends"
tokenized_comment = word_tokenize(sentence)
print("Tokenized comment: ",tokenized_comment)

processed_comment = [word for word in tokenized_comment if word.lower() not in stop_words]
print(processed_comment)

In [ ]:
def remove_stopwords(raw_comment):
  tokenized_comment = word_tokenize(raw_comment)
  processed_comment = [word for word in tokenized_comment if word.lower() not in stop_words]
  return ' '.join(processed_comment)


In [ ]:
result = remove_stopwords( "Today is a very warm and sunny day, I would like to go out and play football with my friends")
print(result)

In [ ]:
#whatever you are trying to return - for - condition
fruits = ["Banana", "Apple","Fig","Orange" ]
result = [fruit for fruit in fruits if len(fruit) < 4]
print(result)


# Using Model :-

In [ ]:
import nltk
#from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt_tab')

In [ ]:
from transformers import pipeline

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

sia = SentimentIntensityAnalyzer()
stop_words = stop_words = set(stopwords.words('english'))
classifier = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")


In [ ]:
import pandas as pd
df = pd.read_excel("/content/SnowWhite Comments.xlsx")

In [ ]:
comments = []
comments = df["Comments"].tolist()

In [ ]:
def remove_stopwords(raw_comment):
  tokenized_comment = word_tokenize(raw_comment)
  processed_comment = [word for word in tokenized_comment if word.lower() not in stop_words]
  return ' '.join(processed_comment)

In [ ]:
def get_comment_sentiment_details(raw_comment):
  processed_comment = remove_stopwords(raw_comment)

  words = processed_comment.split()
  positive_words = ""
  negative_words = ""
  comment_sentiment = "" #Either POSITIVE or NEGATIVE

  sentence_score_temp = sia.polarity_scores(processed_comment)

  abs_sentence_score = abs(sentence_score_temp['compound']) #absolute means if I have -3.4 -> 3.4
  sentiment_label = classifier(processed_comment)
  comment_sentiment = sentiment_label[0]['label']

  if abs_sentence_score == 0:
    comment_sentiment = "NEUTRAL"

  if comment_sentiment == "NEGATIVE":
    sentence_score = abs_sentence_score * -1
    for word in words:
      word_sentiment = sia.polarity_scores(word)
      if word_sentiment['compound'] < 0:
        negative_words += word + " "

  elif comment_sentiment == "POSITIVE":
    sentence_score = abs_sentence_score
    for word in words:
      word_sentiment = sia.polarity_scores(word)
      if word_sentiment['compound'] > 0:
        positive_words += word + " "
  else:
    sentence_score = abs_sentence_score

  return positive_words, negative_words, sentence_score, comment_sentiment

In [ ]:
positive_words = ""
negative_words = ""
neu_count = 0

pos_values_list = []
neg_values_list = []
avg_pos_score = 0
avg_neg_score = 0

for comment in comments:
  pw, nw, ss, cs = get_comment_sentiment_details(comment)
  positive_words += pw+ " " #storing pw from each comment into our central positive words
  negative_words += nw+ " "

  if cs == "NEGATIVE":
    neg_values_list.append(ss)
  elif cs == "POSITIVE":
    pos_values_list.append(ss)
  else:
    neu_count+=1

try:
  avg_pos_score = sum(pos_values_list) / len(pos_values_list)
  avg_neg_score = sum(neg_values_list) / len(neg_values_list)
except ZeroDivisionError:
  if len(pos_values_list) == 0 or len(neg_values_list) == 0:
    avg_pos_score = 0
    avg_neg_score = 0

final_score = (avg_pos_score + avg_neg_score) / (len(pos_values_list) + len(neg_values_list))




In [ ]:
print(final_score)

In [ ]:
positive_words

In [ ]:
negative_words

In [ ]:
avg_pos_score

In [ ]:
avg_neg_score

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

print("positives")
wordcloud_positive = WordCloud(width=800, height=400, background_color='white').generate(positive_words)

plt.figure(figsize=(10,5))
plt.imshow(wordcloud_positive, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
print("negatives")
wordcloud_negative = WordCloud(width=800, height=400, background_color='white').generate(negative_words)

plt.figure(figsize=(10,5))
plt.imshow(wordcloud_negative, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
import nbformat

filename = "Sentiment_Analyzer.ipynb"  # change to your file name
nb = nbformat.read(filename, as_version=nbformat.NO_CONVERT)

# Remove metadata.widgets if 'state' is missing
if 'widgets' in nb.metadata and 'state' not in nb.metadata['widgets']:
    del nb.metadata['widgets']

nbformat.write(nb, filename)
print("Notebook metadata cleaned.")
